### Question 1

Run docker with the python:3.12.8 image in an interactive mode, use the entrypoint bash.\

What's the version of pip in the image?

!powershell docker run -it --entrypoint bash python:3.12.8

The version is 24.3.1

### Question 2

Given the following docker-compose.yaml, what is the hostname and port that pgadmin should use to connect to the postgres database?

```yaml
services:
  db:
    container_name: postgres
    image: postgres:17-alpine
    environment:
      POSTGRES_USER: 'postgres'
      POSTGRES_PASSWORD: 'postgres'
      POSTGRES_DB: 'ny_taxi'
    ports:
      - '5433:5432'
    volumes:
      - vol-pgdata:/var/lib/postgresql/data

  pgadmin:
    container_name: pgadmin
    image: dpage/pgadmin4:latest
    environment:
      PGADMIN_DEFAULT_EMAIL: "pgadmin@pgadmin.com"
      PGADMIN_DEFAULT_PASSWORD: "pgadmin"
    ports:
      - "8080:80"
    volumes:
      - vol-pgadmin_data:/var/lib/pgadmin  

volumes:
  vol-pgdata:
    name: vol-pgdata
  vol-pgadmin_data:
    name: vol-pgadmin_data

```


PgAdmin should use the hostname db and port 5432 to connect to the postgres database.

### Question 3

During the period of October 1st 2019 (inclusive) and November 1st 2019 (exclusive), how many trips, respectively, happened:

```sql
SELECT 
    CASE
		WHEN trip_distance <= 1 THEN 'Up to 1 mile'                                 
        WHEN trip_distance >1 AND trip_distance <=3 THEN 'Between 1 and 3 miles'
        WHEN trip_distance >3 AND trip_distance <=7 THEN 'Between 3 and 7 miles'
		WHEN trip_distance >7 AND trip_distance <=10 THEN 'Between 7 and 10 miles'
        ELSE 'Over 10 miles'
    END AS distance_range,
    COUNT(*) AS trip_count
FROM public.green_taxi_trips
WHERE DATE(lpep_pickup_datetime) >= '2019-10-01' 
AND DATE(lpep_dropoff_datetime) < '2019-11-01' 
GROUP BY distance_range;
```

| Distance Range          | Trip Count |
|-------------------------|------------|
| Up to 1 mile            | 104802     |
| Between 1 and 3 miles   | 198924     |
| Between 3 and 7 miles   | 109603     |
| Between 7 and 10 miles  | 27678      |
| Over 10 miles           | 35189      |


### Question 4

Which was the pick up day with the longest trip distance? Use the pick up time for your calculations.

```sql

WITH daily_longest_trip AS (
    SELECT 
        CAST(lpep_pickup_datetime AS DATE) AS pickup_date,
        MAX(trip_distance) AS max_trip_distance
    FROM public.green_taxi_trips
    WHERE DATE(lpep_pickup_datetime) >= '2019-10-01'
      AND DATE(lpep_pickup_datetime) < '2019-11-01'
    GROUP BY CAST(lpep_pickup_datetime AS DATE)
)
SELECT pickup_date, max_trip_distance
FROM daily_longest_trip
ORDER BY max_trip_distance DESC
LIMIT 1;

```

| Pickup Date | Max Trip Distance |
|-------------|-------------------|
| 2019-10-31  | 515.89            |


### Question 5

Which were the top pickup locations with over 13,000 in total_amount (across all trips) for 2019-10-18?

```sql
SELECT 
    z."Zone", 
    SUM(t.total_amount) AS total_amount
FROM public.green_taxi_trips t
JOIN public.taxi_zone_lookup z 
    ON t."PULocationID" = z."LocationID"
WHERE DATE(t.lpep_pickup_datetime) = '2019-10-18'
GROUP BY z."Zone"
HAVING SUM(t.total_amount) > 13000
ORDER BY total_amount DESC;

```

| Zone               | Total Amount        |
|--------------------|---------------------|
| East Harlem North  | 18686.68            |
| East Harlem South  | 16797.26            |
| Morningside Heights| 13029.79            |

### Question 6

For the passengers picked up in October 2019 in the zone named "East Harlem North" which was the drop off zone that had the largest tip?

```sql
SELECT zdo."Zone", MAX(tip_amount) AS total_tip

FROM public.green_taxi_trips  g
JOIN public.taxi_zone_lookup zpu ON g."PULocationID"  = zpu."LocationID"
JOIN public.taxi_zone_lookup zdo ON g."DOLocationID"  = zdo."LocationID"

WHERE CAST(lpep_pickup_datetime AS DATE) <= '2019-10-31'
AND zpu."Zone" = 'East Harlem North'

GROUP BY zdo."Zone"
ORDER BY total_tip DESC
LIMIT 1

```

| Dropoff Zone   | Total Tip |
|--------------- |-----------|
| JFK Airport    | 87.3      |


### Question 7

Which of the following sequences, respectively, describes the workflow for:

Downloading the provider plugins and setting up backend,\
Generating proposed changes and auto-executing the plan\
Remove all resources managed by terraform

The answer would be terraform init, terraform apply -auto-approve, terraform destroy which would be option 4
